In [1]:
%cd drive/My Drive

/content/drive/My Drive


In [2]:
%cd GOOGLE-COLAB/data/fake-news/

/content/drive/My Drive/GOOGLE-COLAB/data/fake-news


In [3]:
#!pip install nltk

#### Import Libraries

In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwords.words('english')[:5]

['i', 'me', 'my', 'myself', 'we']

In [7]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [9]:
df = df.dropna()

In [10]:
df.isnull().sum()


id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
X = df.drop('label',axis=1)
X.shape

(18285, 4)

In [12]:
y = df['label']
y.shape

(18285,)

In [13]:
vocab_size = 5000

In [14]:
messages = X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [16]:
messages.reset_index(inplace=True)
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [17]:
english_stopwords = stopwords.words('english')
ps = PorterStemmer()

In [18]:
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
print(len(corpus))
corpus[:5]

18285


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [37]:
one_hot_repr = [one_hot(words,vocab_size) for words in corpus]
one_hot_repr[:5]

[[369, 4245, 3935, 2135, 393, 936, 2977, 2663, 3182, 897],
 [2849, 365, 3573, 22, 861, 2007, 2544],
 [3809, 2621, 2185, 2451],
 [4258, 2498, 1186, 660, 805, 1328],
 [4203, 861, 3130, 2912, 1433, 4243, 861, 1676, 749, 4606]]

In [21]:
sent_length=20
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
embedded_docs[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  369,
        4245, 3935, 2135,  393,  936, 2977, 2663, 3182,  897],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2849,  365, 3573,   22,  861, 2007, 2544],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3809, 2621, 2185, 2451],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4258, 2498, 1186,  660,  805, 1328],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4203,
         861, 3130, 2912, 1433, 4243,  861, 1676,  749, 4606]],
      dtype=int32)

### Build Model

In [53]:
embedding_features = 40
model  = Sequential([
                     layers.Embedding(vocab_size,embedding_features,input_length=sent_length),
                     layers.LSTM(100),
                     layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [44]:
# embedding_vector_features=40
# model=Sequential()
# model.add(layers.Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
# model.add(layers.LSTM(100))
# model.add(layers.Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
len(embedded_docs),y.shape

(18285, (18285,))

In [55]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [56]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [58]:
X_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 14], dtype=int32)

In [59]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 15ms/step - loss: 0.3434 - accuracy: 0.8318 - val_loss: 0.1999 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1435 - accuracy: 0.9422 - val_loss: 0.1874 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1040 - accuracy: 0.9612 - val_loss: 0.2133 - val_accuracy: 0.9223
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0715 - accuracy: 0.9750 - val_loss: 0.2457 - val_accuracy: 0.9145
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0453 - accuracy: 0.9847 - val_loss: 0.3470 - val_accuracy: 0.9191
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0261 - accuracy: 0.9922 - val_loss: 0.3248 - val_accuracy: 0.9211
Epoch 7/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0179 - accuracy: 0.9944 - val_loss: 0.3977 - val_accuracy: 0.9142

In [64]:
float(5.3903268e-04)

0.00053903268

In [60]:
model.predict(X_test)[:10]

array([[9.9999940e-01],
       [5.3903268e-04],
       [3.6979457e-03],
       [6.2887067e-01],
       [1.4554793e-03],
       [9.9999893e-01],
       [9.8841095e-01],
       [9.9999893e-01],
       [9.9999976e-01],
       [9.9999106e-01]], dtype=float32)

In [61]:
model.predict_classes(X_test)[:10]

array([[1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [62]:
y_test[:10]

array([1, 0, 0, 0, 1, 1, 0, 1, 1, 1])

In [63]:
confusion_matrix(y_test,model.predict_classes(X_test))

array([[3135,  284],
       [ 195, 2421]])

### Adding Dropout Layers

In [81]:
model_new  = Sequential([
                     layers.Embedding(vocab_size,embedding_features,input_length=sent_length),
                     layers.Dropout(0.3),
                     layers.LSTM(100),
                     layers.Dropout(0.3),
                     layers.Dense(1,activation='sigmoid')
])
model_new.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_new.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [82]:
model_new.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 15ms/step - loss: 0.3572 - accuracy: 0.8312 - val_loss: 0.1992 - val_accuracy: 0.9122
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1507 - accuracy: 0.9406 - val_loss: 0.1892 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1053 - accuracy: 0.9606 - val_loss: 0.2239 - val_accuracy: 0.9011
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0805 - accuracy: 0.9709 - val_loss: 0.2381 - val_accuracy: 0.9158
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0607 - accuracy: 0.9781 - val_loss: 0.2714 - val_accuracy: 0.9054
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0409 - accuracy: 0.9861 - val_loss: 0.2632 - val_accuracy: 0.9105
Epoch 7/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0301 - accuracy: 0.9901 - val_loss: 0.4166 - val_accuracy: 0.9171

In [83]:
model_new.predict(X_test)

array([[9.9993730e-01],
       [4.4966633e-08],
       [2.6655386e-04],
       ...,
       [1.1677096e-06],
       [9.9034041e-01],
       [9.8713350e-01]], dtype=float32)

### Bidirection Model

In [84]:
model_new  = Sequential([
                     layers.Embedding(vocab_size,embedding_features,input_length=sent_length),
                     layers.Bidirectional(layers.LSTM(100)),
                     layers.Dropout(0.3),
                     layers.Dense(1,activation='sigmoid')
])
model_new.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_new.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [85]:
model_new.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3045 - accuracy: 0.8536 - val_loss: 0.2164 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1387 - accuracy: 0.9465 - val_loss: 0.2027 - val_accuracy: 0.9130
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0903 - accuracy: 0.9681 - val_loss: 0.2241 - val_accuracy: 0.9196
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0598 - accuracy: 0.9784 - val_loss: 0.2749 - val_accuracy: 0.9153
Epoch 5/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0395 - accuracy: 0.9878 - val_loss: 0.3713 - val_accuracy: 0.9100
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0268 - accuracy: 0.9917 - val_loss: 0.3966 - val_accuracy: 0.9132
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0157 - accuracy: 0.9952 - val_loss: 0.5090 - val_accuracy: 0.9080

In [86]:
model_new.predict(X_test)

array([[9.9998677e-01],
       [2.6629319e-05],
       [2.4346772e-03],
       ...,
       [1.6078680e-04],
       [9.9981052e-01],
       [9.9887520e-01]], dtype=float32)